In [1]:
import pandas as pd
import numpy as np
import transformers as trf
from datasets import Dataset
import torch
from accelerate import Accelerator
from accelerate import notebook_launcher

In [2]:
import os
os.chdir("/home/573/rh2942/WASSA-2023-EMP") # changing dir for evaluation file
# print(os.getcwd())

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false" # due to huggingface warning

In [4]:
def load_tokenised_data(filename, tokeniser, train_test):
   
    input_data = pd.read_csv(filename, header=0, index_col=0)
    
    if train_test == "train":
        chosen_data = input_data[[feature_1, feature_2, task]]
    elif train_test == "test":
        chosen_data = input_data[[feature_1, feature_2]]  #test data shouldn't have output label

    hugging_dataset = Dataset.from_pandas(chosen_data, preserve_index=False)

    tokenised_hugging_dataset = hugging_dataset.map(tokenise, batched=True, remove_columns = [feature_1, feature_2])
    
    if train_test == "train":
        tokenised_hugging_dataset = tokenised_hugging_dataset.rename_column(task, "labels") # as huggingface requires
    
    tokenised_hugging_dataset = tokenised_hugging_dataset.with_format("torch")

    return tokenised_hugging_dataset

# Prediction

In [14]:
def train_test(model):
    accelerator = Accelerator()
    
    accelerator.print(f"{task} prediction")
    
    opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
  
    trainset = load_tokenised_data(filename=os.path.join("./processed_data", train_filename), tokeniser=tokeniser, train_test="train")
       
    trainloader = torch.utils.data.DataLoader(
        trainset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
    )
    
    training_steps = NUM_EPOCH * len(trainloader)
    lr_scheduler = trf.get_scheduler(
        "linear",
        optimizer=opt,
        num_warmup_steps=0,
        num_training_steps=training_steps
    )

    trainloader, model, opt = accelerator.prepare(
        trainloader, model, opt    
    )
    
    for epoch in range(0, NUM_EPOCH):

        # Print epoch
        accelerator.print(f'Starting epoch {epoch+1}')
        
        epoch_loss = 0
        num_batches = 0

        # Iterate over the DataLoader for training data
        for batch in trainloader:
            # Perform forward pass
            outputs = model(**batch)
            
            loss = outputs.loss

            accelerator.backward(loss)
        
            opt.step()
            lr_scheduler.step()
            
            opt.zero_grad()
            
            epoch_loss += loss.item()
            num_batches += 1

        # Process is complete.
        avg_epoch_loss = epoch_loss / num_batches
        accelerator.print(f"Epoch {epoch}: average loss = {avg_epoch_loss}")
        
    
    # evaluation on test set
    testset = load_tokenised_data(filename=os.path.join("./processed_data", test_filename), tokeniser=tokeniser, train_test="test")
    testloader = torch.utils.data.DataLoader(
        testset, shuffle=False, batch_size=BATCH_SIZE, collate_fn=data_collator
    )
   
    model.eval()

    y_pred = []

    for batch in testloader:
        with torch.no_grad():
            outputs = model(**batch)

        batch_pred = [item for sublist in outputs.logits.tolist() for item in sublist]  #convert 2D list to 1D
        y_pred.extend(batch_pred)
  
    y_pred_df = pd.DataFrame({task: y_pred})
    filename = "./prediction/predictions_" + task + ".tsv"
    y_pred_df.to_csv(filename, sep='\t', header=False, index=False)

In [15]:
NUM_EPOCH = 15
BATCH_SIZE = 2
LEARNING_RATE = 5e-5

train_filename = "preprocessed_train.csv"
test_filename = "preprocessed_dev.csv"
# test_filename = "preprocessed_test.csv"

#Chosen features
feature_1 = 'demographic_essay'
feature_2 = 'article'
    
# checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased"

tokeniser = trf.AutoTokenizer.from_pretrained(checkpoint)

#padding="longest" can be deferred to do dynamic padding
def tokenise(sentence):
    return tokeniser(sentence[feature_1], sentence[feature_2], truncation=True)
    
# data collator due to variable max token length per batch size
data_collator = trf.DataCollatorWithPadding(tokenizer = tokeniser)

In [16]:
model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
task = 'personality_conscientiousness'
notebook_launcher(train_test, (model,), num_processes=torch.cuda.device_count())
predictions_cons = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Launching training on 4 GPUs.
personality_conscientiousness prediction


Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Starting epoch 1


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 3.6553235433482545
Starting epoch 2
Epoch 1: average loss = 0.7847215380764339
Starting epoch 3
Epoch 2: average loss = 0.23317860238128152
Starting epoch 4
Epoch 3: average loss = 0.0401560267774232
Starting epoch 5
Epoch 4: average loss = 0.012054131746079905
Starting epoch 6
Epoch 5: average loss = 0.007825922980883864
Starting epoch 7
Epoch 6: average loss = 0.005224421850271585
Starting epoch 8
Epoch 7: average loss = 0.002731663824742638
Starting epoch 9
Epoch 8: average loss = 0.0020219572540127605
Starting epoch 10
Epoch 9: average loss = 0.0007563634978804639
Starting epoch 11
Epoch 10: average loss = 0.0007357247656672167
Starting epoch 12
Epoch 11: average loss = 0.0009886354438322087
Starting epoch 13
Epoch 12: average loss = 0.0010136106877919758
Starting epoch 14
Epoch 13: average loss = 0.001156120645349319
Starting epoch 15
Epoch 14: average loss = 0.0015039569362283126


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [17]:
model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
task = 'personality_openess'
notebook_launcher(train_test, (model,), num_processes=torch.cuda.device_count())
predictions_open = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Launching training on 4 GPUs.
personality_openess prediction


Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Starting epoch 1


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 3.2126965843977366
Starting epoch 2
Epoch 1: average loss = 1.2351526914506543
Starting epoch 3
Epoch 2: average loss = 0.27538824014897856
Starting epoch 4
Epoch 3: average loss = 0.25245657728834087
Starting epoch 5
Epoch 4: average loss = 0.07352666695464441
Starting epoch 6
Epoch 5: average loss = 0.0390422332874346
Starting epoch 7
Epoch 6: average loss = 0.018534863260801827
Starting epoch 8
Epoch 7: average loss = 0.013046674841424813
Starting epoch 9
Epoch 8: average loss = 0.007043037601649804
Starting epoch 10
Epoch 9: average loss = 0.005494033035686197
Starting epoch 11
Epoch 10: average loss = 0.006486802380763669
Starting epoch 12
Epoch 11: average loss = 0.0029365332715389405
Starting epoch 13
Epoch 12: average loss = 0.004135310347553003
Starting epoch 14
Epoch 13: average loss = 0.00273951646958109
Starting epoch 15
Epoch 14: average loss = 0.002075402805113132


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [18]:
model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
task = 'personality_extraversion'
notebook_launcher(train_test, (model,), num_processes=torch.cuda.device_count())
predictions_extr = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Launching training on 4 GPUs.
personality_extraversion prediction


Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Starting epoch 1


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 3.6351895903056604
Starting epoch 2
Epoch 1: average loss = 0.4150892763674183
Starting epoch 3
Epoch 2: average loss = 0.20990150685752362
Starting epoch 4
Epoch 3: average loss = 0.15876231847891623
Starting epoch 5
Epoch 4: average loss = 0.05067670642339478
Starting epoch 6
Epoch 5: average loss = 0.017482588375691083
Starting epoch 7
Epoch 6: average loss = 0.013827559836982448
Starting epoch 8
Epoch 7: average loss = 0.006512351197448063
Starting epoch 9
Epoch 8: average loss = 0.005411444063731918
Starting epoch 10
Epoch 9: average loss = 0.004030455063068716
Starting epoch 11
Epoch 10: average loss = 0.002774560444734385
Starting epoch 12
Epoch 11: average loss = 0.0037730757687443528
Starting epoch 13
Epoch 12: average loss = 0.002895549578856937
Starting epoch 14
Epoch 13: average loss = 0.002955134549449899
Starting epoch 15
Epoch 14: average loss = 0.002590018765064941


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [19]:
model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
task = 'personality_agreeableness'
notebook_launcher(train_test, (model,), num_processes=torch.cuda.device_count())
predictions_agre = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Launching training on 4 GPUs.
personality_agreeableness prediction


Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Starting epoch 1


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 3.784195236419803
Starting epoch 2
Epoch 1: average loss = 0.645002929720676
Starting epoch 3
Epoch 2: average loss = 0.17074609287433584
Starting epoch 4
Epoch 3: average loss = 0.08279661837009278
Starting epoch 5
Epoch 4: average loss = 0.025994332882855443
Starting epoch 6
Epoch 5: average loss = 0.012780121475603961
Starting epoch 7
Epoch 6: average loss = 0.0047820038475854705
Starting epoch 8
Epoch 7: average loss = 0.005713648862967825
Starting epoch 9
Epoch 8: average loss = 0.0034144652852551283
Starting epoch 10
Epoch 9: average loss = 0.0029730841773141724
Starting epoch 11
Epoch 10: average loss = 0.0018015422936384442
Starting epoch 12
Epoch 11: average loss = 0.0013845333734971805
Starting epoch 13
Epoch 12: average loss = 0.0009325081132931013
Starting epoch 14
Epoch 13: average loss = 0.001193727776307223
Starting epoch 15
Epoch 14: average loss = 0.0011290962035431851


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [22]:
model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
task = 'personality_stability'
notebook_launcher(train_test, (model,), num_processes=torch.cuda.device_count())
predictions_stab = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

Launching training on 4 GPUs.
personality_stability prediction


Map:   0%|          | 0/779 [00:00<?, ? examples/s]

Starting epoch 1


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 4.718034930130448
Starting epoch 2
Epoch 1: average loss = 1.1397955859916247
Starting epoch 3
Epoch 2: average loss = 0.3495844700326193
Starting epoch 4
Epoch 3: average loss = 0.08943492133789269
Starting epoch 5
Epoch 4: average loss = 0.050457261532254945
Starting epoch 6
Epoch 5: average loss = 0.06695095978147018
Starting epoch 7
Epoch 6: average loss = 0.042801451445670026
Starting epoch 8
Epoch 7: average loss = 0.02810986917081339
Starting epoch 9
Epoch 8: average loss = 0.005235713211335474
Starting epoch 10
Epoch 9: average loss = 0.005136327689115205
Starting epoch 11
Epoch 10: average loss = 0.0017815178103050204
Starting epoch 12
Epoch 11: average loss = 0.000980624836863487
Starting epoch 13
Epoch 12: average loss = 0.0008388755061843062
Starting epoch 14
Epoch 13: average loss = 0.0006430275922114987
Starting epoch 15
Epoch 14: average loss = 0.0006185626143656473


Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

In [24]:
predictions_PER = pd.concat([predictions_cons, predictions_open, predictions_extr, predictions_agre, predictions_stab], axis=1)

predictions_PER.to_csv("./prediction/predictions_PER.tsv", sep='\t', header=False, index=False)

In [3]:
from evaluation import pearsonr, calculate_pearson

In [7]:
# # Just checking the dev set performance
gold_dev = pd.read_csv('./dataset/dev/goldstandard_dev.tsv', sep='\t', header=None) # no header
predictions_PER= pd.read_csv("./prediction/predictions_PER.tsv", sep='\t', header=None)

print(f"cons: {pearsonr(gold_dev.loc[:,3].tolist(), predictions_PER.loc[:,0].tolist())}")
print(f"open: {pearsonr(gold_dev.loc[:,4].tolist(), predictions_PER.loc[:,1].tolist())}")
print(f"extr: {pearsonr(gold_dev.loc[:,5].tolist(), predictions_PER.loc[:,2].tolist())}")
print(f"agre: {pearsonr(gold_dev.loc[:,6].tolist(), predictions_PER.loc[:,3].tolist())}")
print(f"stab: {pearsonr(gold_dev.loc[:,7].tolist(), predictions_PER.loc[:,4].tolist())}")

cons: 0.6117
open: 0.6471
extr: 0.5229
agre: 0.1422
stab: 0.2999


# Hyperparam tuning

In [6]:
import csv
from sklearn.model_selection import KFold
from evaluation import pearsonr, calculate_pearson

In [12]:
def train_test_kfold(model):
    accelerator = Accelerator()
    
    opt = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    training_steps = NUM_EPOCH * len(trainloader)
    lr_scheduler = trf.get_scheduler(
        "linear",
        optimizer=opt,
        num_warmup_steps=0,
        num_training_steps=training_steps
    )
  
    trainloader_acclerate, model, opt = accelerator.prepare(
        trainloader, model, opt    
    )  
    
    for epoch in range(0, NUM_EPOCH):

        # Print epoch
        accelerator.print(f'Starting epoch {epoch+1}')
        
        epoch_loss = 0
        num_batches = 0

        # Iterate over the DataLoader for training data
        for batch in trainloader_acclerate:
            # Perform forward pass
            outputs = model(**batch)
            
            loss = outputs.loss
#             loss = loss_function(outputs, targets)

            accelerator.backward(loss)
        
            opt.step()
            lr_scheduler.step()
            
            opt.zero_grad()
            
            epoch_loss += loss.item()
            num_batches += 1

        # Process is complete.
        avg_epoch_loss = epoch_loss / num_batches
        accelerator.print(f"Epoch {epoch}: average loss = {avg_epoch_loss}")
    
    
    # Evaluation
        
    model.eval()

    y_true =[]
    y_pred = []

    for batch in testloader:
        with torch.no_grad():
            outputs = model(**batch)

        y_true.extend(batch['labels'].tolist())

        batch_pred = [item for sublist in outputs.logits.tolist() for item in sublist]  #convert 2D list to 1D
        y_pred.extend(batch_pred)
        
    pearson_r = pearsonr(y_true, y_pred)
    
    accelerator.print('\n' + checkpoint + ' & ' + str(LEARNING_RATE) + ' & ' + str(BATCH_SIZE) + ' & ' + feature_1 + '-' + feature_2 + ' & ' + str(pearson_r) + ' fold-' + str(fold) + '\n')

In [72]:
NUM_EPOCH = 10
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

task = "empathy"

train_dev_filename = "preprocessed_train_dev.csv"

#Chosen features
feature_1 = 'demographic_essay'
feature_2 = 'article'

# feature_1 = 'essay'
# feature_2 = 'demographic'

# feature_1 = 'essay'
# feature_2 ='article'
    
# checkpoint = "bert-base-uncased"
# checkpoint = "distilbert-base-uncased"

tokeniser = trf.AutoTokenizer.from_pretrained(checkpoint)

#padding="longest" can be deferred to do dynamic padding
def tokenise(sentence):
    return tokeniser(sentence[feature_1], sentence[feature_2], truncation=True)
    
# data collator due to variable max token length per batch size
data_collator = trf.DataCollatorWithPadding(tokenizer = tokeniser)

train_dev = load_tokenised_data(filename=os.path.join("./processed_data", train_dev_filename), tokeniser=tokeniser, train_test="train")

K_FOLD = 5

# Set fixed random number seed
torch.manual_seed(42)

kfold = KFold(n_splits=K_FOLD, shuffle=True)

print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_idx, test_idx) in enumerate(kfold.split(train_dev)):

    print(f'FOLD {fold}')
    print('--------------------------------')

    # Sample elements randomly from a given list of ids, no replacement.
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_idx)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(
        train_dev,
        batch_size=BATCH_SIZE,
        collate_fn=data_collator,
        sampler=train_subsampler
    )
    testloader = torch.utils.data.DataLoader(
        train_dev,
        batch_size=BATCH_SIZE,
        collate_fn=data_collator,
        sampler=test_subsampler
    )

    # Init the neural network
    model = trf.AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)

    notebook_launcher(train_test_kfold, (model,), num_processes=torch.cuda.device_count())

Map:   0%|          | 0/987 [00:00<?, ? examples/s]

--------------------------------
FOLD 0
--------------------------------


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Launching training on 4 GPUs.
Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Epoch 0: average loss = 5.486358122825623
Starting epoch 2
Epoch 1: average loss = 3.3773480463027954
Starting epoch 3
Epoch 2: average loss = 3.858164029121399
Starting epoch 4
Epoch 3: average loss = 3.092499678134918
Starting epoch 5
Epoch 4: average loss = 2.3774873328208925
Starting epoch 6
Epoch 5: average loss = 2.290995116233826
Starting epoch 7
Epoch 6: average loss = 2.1909004259109497
Starting epoch 8
Epoch 7: average loss = 1.3470318806171417
Starting epoch 9
Epoch 8: average loss = 0.9972421827912331
Starting epoch 10
Epoch 9: average loss = 0.7596608233451844

bert-large-uncased & 5e-05 & 8 & demographic_essay-article & 0.7008 fold-3

FOLD 4
--------------------------------


Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Launching training on 4 GPUs.
Starting epoch 1


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 0: average loss = 8.129480237960815
Starting epoch 2
Epoch 1: average loss = 3.9551459836959837
Starting epoch 3
Epoch 2: average loss = 3.8354076099395753
Starting epoch 4
Epoch 3: average loss = 3.8489742612838747
Starting epoch 5
Epoch 4: average loss = 5.5751168823242185
Starting epoch 6
Epoch 5: average loss = 3.7704463481903074
Starting epoch 7
Epoch 6: average loss = 3.774736123085022
Starting epoch 8
Epoch 7: average loss = 3.518358378410339
Starting epoch 9
Epoch 8: average loss = 3.6325153732299804
Starting epoch 10
Epoch 9: average loss = 3.7241961479187013

bert-large-uncased & 5e-05 & 8 & demographic_essay-article & -0.035 fold-4

